In [ ]:
import os
import shutil
from lumpyrem import run

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # Suppress deprecation warnings

In [ ]:
org_model_ws = os.path.join('..', 'temp_ml_obs')
os.listdir(org_model_ws)

In [ ]:
tmp_model_ws = os.path.join('..', 'temp_ml_calib') # Safe to delete
if os.path.exists(tmp_model_ws):
    shutil.rmtree(tmp_model_ws)
shutil.copytree(org_model_ws,tmp_model_ws)
os.listdir(tmp_model_ws)

In [ ]:
pest_ws = os.path.join(tmp_model_ws, 'pest')

In [ ]:
def write_script(filename, lines):

    with open(filename, 'a') as f:
        for line in lines:
            f.write(line)
            f.write('\n')

In [ ]:
write_script(os.path.join(pest_ws, 'runmodel.bat'), [
'''
@echo off
Rem ###################################
Rem Some intermediate files are deleted as a precaution.
Rem ###################################
cd %~dp0..\\runmodel\\model
del /s *.csv

Rem ###################################
Rem PLPROC assigns parameters from ppoints
Rem ###################################
cd %~dp0..\\runmodel\\preproc\\
plproc plproc.dat
plproc plproc_sfr.dat
plproc plproc_ghb.dat

Rem ###################################
Rem The MF6 model is run
Rem ###################################
cd %~dp0..\\runmodel\\model\\
mf6 mfsim.nam

Rem ###################################
Rem The observations are postprocessed
Rem ###################################
cd  %~dp0..\\runmodel\\postproc\\
olproc olproc.in 1

Rem ###################################
Rem Return the command line to the working folder
Rem ###################################
cd %~dp0.\\
'''
])

### Make copy of calibration file

In [ ]:
os.listdir(tmp_model_ws)

In [ ]:
with open(os.path.join(pest_ws, 'calib0.pst'), 'r') as file:
    calib1 = file.readlines()

### Change NOPTMAX from 50 to 0:

In [ ]:
temp_row = calib1[8].split()
display(temp_row)

In [ ]:
temp_row[0] = '0'
temp_row[-1] = f'{temp_row[-1]}\n'
display(temp_row)

In [ ]:
new_row = ' '.join(temp_row)
display(new_row)

In [ ]:
calib1[8] = new_row

### Add model command line

In [ ]:
mlcl_strt = calib1.index('* model command line\n') + 1
mlcl_stop = calib1.index('* model input/output\n')

In [ ]:
calib1[mlcl_strt:mlcl_stop] = 'runmodel.bat\n'

### Write file

In [ ]:
# Write the file out again
with open(os.path.join(pest_ws, 'calib1.pst'), 'w') as file:
    for line in calib1:
        file.write(line)

## Run model once manually (try to script) ##

run process does not seem to work with tpl2pst, tempchek, pestchek, pest_hp, pwtadj1

In [ ]:
import subprocess
p = subprocess.run(['pwtadj1', 'calib1.pst', 'calib1-wt.pst', '1000'], cwd=pest_ws, stdout=subprocess.PIPE, text=True)
for row in p.stdout.split('\n'):
    print(row)

### Adding Tikhonov Regularization

In [ ]:
p = subprocess.run(['addreg1', 'calib1-wt.pst', 'calib1-wt-reg.pst'], cwd=pest_ws, stdout=subprocess.PIPE, text=True)

In [ ]:
p.stdout.split('\n')

In [ ]:
p = subprocess.run(['pestchek', 'calib1-wt-reg.pst'], cwd=pest_ws, stdout=subprocess.PIPE, text=True)
for row in p.stdout.split('\n'):
    print(row)

#### Add covariance matrices

In [ ]:
write_script(os.path.join(pest_ws, 'covmatfile.in'), [
'''
regul_kx1 ..\runmodel\preproc\cov1.mat
regul_kx2 ..\runmodel\preproc\cov2.mat
regul_kx3 ..\runmodel\preproc\cov3.mat
regul_rch ..\runmodel\preproc\cov_rch.mat
regul_strc ..\runmodel\preproc\cov_stream.mat
regul_ghb1 ..\runmodel\preproc\cov_ghb.mat
regul_ghb3 ..\runmodel\preproc\cov_ghb.mat

regul_ghbcbl
regul_ghbcli
regul_ghbcma
regul_ghbcor
regul_ghbcre
regul_ghbcro
regul_ghbcye
regul_kh1pp
regul_kh2pp
regul_kh3pp
regul_kv1pp
regul_kv2pp
regul_kv3pp
regul_sfrpp
regul_ss1pp
regul_ss2pp
regul_ss3pp
regul_sy1pp
regul_sy2pp
regul_sy3pp
'''
])

In [ ]:
p = subprocess.run(['addcovmat', 'calib1-wt-reg.pst', 'covmatfile.in', 'calib1-wt-cov.pst'], cwd=pest_ws, stdout=subprocess.PIPE, text=True)
for row in p.stdout.split('\n'):
    print(row)

### Set UPTESTMIN to 20
When using PEST_HP it is recommended that the UPTESTMIN variable be set to 20 or higher. This ensures that PEST_HP carries out at least 20 model runs when testing parameter upgrades. If PEST_HP has more than 20 CPU cores at its disposal, it will automatically use all of them to carry out more lambda-testing model runs.

If iteration-specific inversion results are saved, a user can retrieve parameter values obtained at the end of the iteration at which the objective function started to flatten out, but at which the parameter field was more reasonable. Adding the PARSAVEITN and REISAVEITN variables to the “control data” section of a control file activates this behaviour

run process does not seem to work with tpl2pst, tempchek, pestchek, pest_hp, pwtadj1

In [ ]:
# Write the file out again
with open(os.path.join(pest_ws, 'calib1.pst'), 'w') as file:
    file.write(filedata)